In [1]:
import pandas as pd
import os

# 데이터 디렉토리
data_dir = "/home/jovyan/work/data"

all_data = []

# 각 테이블에서 데이터 추출
tables = [
    ('accounts_blockrecord_processed.csv', 'user_id'),
    ('accounts_failpaymenthistory.csv', 'user_id'),
    ('accounts_paymenthistory.csv', 'user_id'),
    ('accounts_pointhistory.csv', 'user_id'),
    ('accounts_timelinereport.csv', 'user_id'),
    ('accounts_user_processed.csv', 'id'),  # id가 user_id 역할
    ('accounts_userquestionrecord_processed.csv', 'user_id'),
    ('event_receipts.csv', 'user_id'),
    ('polls_questionreport.csv', 'user_id'),
    ('polls_questionset.csv', 'user_id')
]

# 일반 테이블들 처리
for file_name, user_col in tables:
    df = pd.read_csv(os.path.join(data_dir, file_name))
    temp_df = df[[user_col, 'created_at']].copy()
    temp_df.columns = ['user_id', 'created_at']
    temp_df['table_name'] = file_name.replace('.csv', '')
    all_data.append(temp_df)

# 친구 요청 테이블 - 보낸 사람만 포함 (능동적 활동)
df_friend = pd.read_csv(os.path.join(data_dir, 'accounts_friendrequest.csv'))

# 보낸 사람만 (능동적 활동)
temp_send = df_friend[['send_user_id', 'created_at']].copy()
temp_send.columns = ['user_id', 'created_at']
temp_send['table_name'] = 'accounts_friendrequest_send'
all_data.append(temp_send)

# 모든 데이터 병합
merged_df = pd.concat(all_data, ignore_index=True)

# created_at을 datetime으로 변환
merged_df['created_at'] = pd.to_datetime(merged_df['created_at'], format='ISO8601', errors='coerce')

# null 값 제거
merged_df = merged_df.dropna(subset=['created_at'])

# user_id로 그룹화하고 created_at으로 정렬
merged_df = merged_df.sort_values(['user_id', 'created_at']).reset_index(drop=True)

print(f"총 {len(merged_df):,}개 레코드 병합 완료")
print(f"유니크 사용자 수: {merged_df['user_id'].nunique():,}")
print("\n테이블별 레코드 수:")
print(merged_df['table_name'].value_counts())

총 21,705,813개 레코드 병합 완료
유니크 사용자 수: 677,085

테이블별 레코드 수:
table_name
accounts_friendrequest_send              17147175
accounts_pointhistory                     2338918
accounts_userquestionrecord_processed     1217558
accounts_user_processed                    677085
polls_questionset                          158384
accounts_paymenthistory                     95140
polls_questionreport                        51424
accounts_blockrecord_processed              19449
event_receipts                                309
accounts_timelinereport                       208
accounts_failpaymenthistory                   163
Name: count, dtype: int64


In [2]:
import pandas as pd
import os

# 경로 설정
data_dir = '/home/jovyan/work/data'

# 포인트 이벤트 테이블
events = pd.read_csv(os.path.join(data_dir, 'events.csv'))

# 출석 테이블
accounts_attendance = pd.read_csv(os.path.join(data_dir, 'accounts_attendance.csv'))

# 친구 요청 테이블
accounts_friendrequest = pd.read_csv(os.path.join(data_dir, 'accounts_friendrequest.csv'))

# 학급 테이블
accounts_group = pd.read_csv(os.path.join(data_dir, 'accounts_group.csv'))

# 가까운 학교를 기록해두기 위한 관계형 테이블
accounts_nearbyschool = pd.read_csv(os.path.join(data_dir, 'accounts_nearbyschool.csv'))

# 유저 컨택 테이블
accounts_user_contacts = pd.read_csv(os.path.join(data_dir, 'accounts_user_contacts.csv'))

# 학교 테이블
accounts_school = pd.read_csv(os.path.join(data_dir, 'accounts_school.csv'))

# 포인트 이벤트 참여 테이블
accounts_userwithdraw = pd.read_csv(os.path.join(data_dir, 'accounts_userwithdraw.csv'))

# 질문 내용 테이블
polls_question = pd.read_csv(os.path.join(data_dir, 'polls_question.csv'))

# 질문에 대한 신고 기록 테이블
polls_questionpiece = pd.read_csv(os.path.join(data_dir, 'polls_questionpiece.csv'))

# 질문에 등장하는 유저들 테이블
polls_usercandidate = pd.read_csv(os.path.join(data_dir, 'polls_usercandidate.csv'))

In [8]:
# 사용자별 생존기간 계산
user_survival = merged_df.groupby('user_id').agg({
    'created_at': ['first', 'last', 'count']
}).reset_index()

# 컬럼명 정리
user_survival.columns = ['user_id', 'first_activity', 'last_activity', 'total_events']

print(f"사용자 생존기간 테이블 완성: {len(user_survival):,}명")
print(user_survival.head())

# 생존일 수 계산 (차이 in days)
user_survival['survival_days'] = (user_survival['last_activity'] - user_survival['first_activity']).dt.days

# 이탈 유저 (당일 유저): 1, 잔존 유저: 0
user_survival['is_churn'] = user_survival['survival_days'].apply(lambda x: 1 if x == 0 else 0)

print(user_survival['is_churn'].value_counts())
# 1: 이탈 (하루만 활동), 0: 잔존

사용자 생존기간 테이블 완성: 677,085명
   user_id             first_activity              last_activity  total_events
0   831956 2023-03-29 03:44:14.047130 2023-03-29 03:44:14.047130             1
1   831962 2023-03-29 05:18:56.162368 2023-07-13 07:51:59.000000             2
2   832151 2023-03-29 12:56:34.989468 2023-05-09 15:50:58.000000            11
3   832340 2023-03-29 12:56:35.020790 2023-05-11 00:51:26.000000            29
4   832520 2023-03-29 12:56:35.049311 2023-03-29 12:56:35.049311             1
is_churn
0    441629
1    235456
Name: count, dtype: int64


In [4]:
# merged_df에서 질문 참여 로그만 추출
question_logs = merged_df[merged_df['table_name'] == 'accounts_userquestionrecord_processed']
question_users_merged = question_logs['user_id'].dropna().unique()



In [ ]:
# 질문에 등장하는 유저 추출
question_users_candidates = polls_usercandidate['user_id'].dropna().unique()


In [6]:
# 통합 질문 유저 목록
question_users_total = set(question_users_merged).union(set(question_users_candidates))


In [9]:
# 플래그 추가
user_survival['has_question_experience'] = user_survival['user_id'].isin(question_users_total)


In [10]:
# 분포 확인
cross_tab = pd.crosstab(user_survival['has_question_experience'], user_survival['is_churn'])
cross_tab.index = ['No Question Exp.', 'Has Question Exp.']
cross_tab.columns = ['Retained (0)', 'Churned (1)']
print(cross_tab)


                   Retained (0)  Churned (1)
No Question Exp.         426237       230845
Has Question Exp.         15392         4611


In [ ]:
# 질문 경험이 있는 유저가 잔존율이 더 높음
질문 경험 유무	잔존 유저 (0)	이탈 유저 (1)
❌ 경험 없음	426,237	230,845
✅ 질문 경험 있음	15,392	4,611

질문 경험 유저의 잔존율 : 76/97%
질문 경험 없는 유저의 잔존율 : 64.86%

: 

In [ ]:
from scipy.stats import chi2_contingency

# 교차표를 그대로 사용해도 됨
chi2, p, dof, expected = chi2_contingency([
    [426237, 230845],
    [15392, 4611]
])

print(f'Chi-square Test p-value: {p:.10f}')
